In [ ]:
%cd "full/path/to/codebase"
import json
from pathlib import Path

from PIL import Image
from tqdm import tqdm

from src.nn.image_feature_extraction import ImageFeatureExtraction

/home/anhvd27/miniforge3/envs/mlqa-tsr/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/anhvd27/miniforge3/envs/mlqa-tsr/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/anhvd27/projects/vlsp/mlqa-tsr/codebase


/home/anhvd27/miniforge3/envs/mlqa-tsr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
image_feature_extraction = ImageFeatureExtraction()

/home/anhvd27/miniforge3/envs/mlqa-tsr/lib/python3.10/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/home/anhvd27/miniforge3/envs/mlqa-tsr/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [ ]:
image_folder = Path("../data/private_test/private_test_images")
image_object_data = json.load(Path("assets/private_test_image_object.json").open("r", encoding="utf-8"))
# do this again with training images

In [4]:
image_feature_list = []

for i in tqdm(image_folder.glob("*.jpg")):
    image = Image.open(i).convert("RGB")
    general_feature = image_feature_extraction.infer_single(image)

    object_feature_list = []
    for obj_idx, bbox in enumerate(image_object_data[i.name]["boxes"]):
        cropped_image = image.crop(bbox)
        # I haven't resized the object according to ImageFeatureExtraction's resized values
        # maybe I don't even need to do that
        if cropped_image.width <= 32 or cropped_image.height <= 32:
            continue
        object_feature = image_feature_extraction.infer_single(cropped_image)
        object_feature_list.append(
            {
                "object_idx": obj_idx,
                "object_feature": object_feature,
            },
        )

    image_feature_list.append(
        {
            "image_name": i.name,
            "general_feature": general_feature,
            "object_feature_list": object_feature_list,
        }
    )
    with open("assets/private_test_image_feature.json", "w", encoding="utf-8") as f:
        json.dump(image_feature_list, f, indent=2, ensure_ascii=False)

104it [00:47,  2.17it/s]
